In [16]:
from google.colab import drive
drive.mount('/content/drive')

import pandas
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
files = os.listdir()
processed_files = 

['.git',
 '.idea',
 'CSC 466 Project Pitch- Group 3.pdf',
 'Lab_6.ipynb',
 'README.md',
 'a.pdf',
 'b.pdf',
 'harsh.py',
 'images',
 'pdfHash.csv',
 'pdfHash.txt',
 'pdfs',
 'python',
 'test.ipynb',
 'testing.py']

In [15]:
%cd /content/drive/MyDrive/466\ Project/CSC466-Project

/content/drive/MyDrive/466 Project/CSC466-Project


In [13]:
!pwd

/content/drive/MyDrive/466 Project


In [6]:
# processed unprocessed files
for file in files and not in processed_files:
  df = pd.read_csv(file)
  df.dtypes

  df.to_csv(new_dir + file)

filename               object
file size             float64
file type              object
file dist of bytes     object
consecutiveByte        object
consecutiveCount        int64
SHA-256                object
SHA-1                  object
MD5                    object
SHA-3-256              object
dtype: object

In [19]:
a = [1,2,3,4]
b = [2,5,6]

for i in b and not in a:
  print(i)

SyntaxError: ignored

# TO DO

* Count total number of chars for each file
* Pick random hash out of available hashes; record both hash and name of hash
* Split hash into groups/chunks (3, maybe 5)
* Count length of hash
* Count number of 0s/1s in hash
* Distribution of 0-e in hash